## Import dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
plt.style.use('ggplot')
%matplotlib inline

In [4]:
df_all = pd.read_csv('data/april_report.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_all.columns = [col.lower().replace(' ','_') for col in df_all.columns.tolist()]
df_all.columns = [col.lower().replace('/timestamp','') for col in df_all.columns.tolist()]

In [6]:
date_col = [col for col in df_all.columns.tolist() if 'date' in col]
df_all[date_col] = df_all[date_col].apply(pd.to_datetime,format = "%m/%d/%Y %H:%M:%S")

In [7]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253306 entries, 0 to 1253305
Data columns (total 23 columns):
unique_row_id           1253306 non-null object
cvo_sid                 1253306 non-null object
event_site_id           1253306 non-null int64
touchpoint_site_id      1253306 non-null int64
event_site_name         1253306 non-null object
touchpoint_site_name    1253306 non-null object
event_type              1253306 non-null object
event_id                1253306 non-null object
event_date              1253306 non-null datetime64[ns]
touchpoint_date         1253306 non-null datetime64[ns]
total_event_value       1253306 non-null float64
touchpoint_revenue      1253306 non-null float64
conversion              1253306 non-null float64
touchpoint_type         1027589 non-null object
first_touch_date        1253306 non-null datetime64[ns]
last_touch_date         1253306 non-null datetime64[ns]
source_1                1253306 non-null object
source_2                1253306 non-nu

## Clean

In [33]:
selected_col=['event_type','conversion','touchpoint_type','source_1','touchpoints_in_path','attribution_sequence','path_position']
selected_col.extend(date_col)

In [34]:
df_model=df_all[selected_col]

In [35]:
df_model = df_model[df_model.source_1!='base']
df_model.reset_index(drop=True, inplace=True)

In [36]:
df_model.head()

,event_type,conversion,touchpoint_type,source_1,touchpoints_in_path,attribution_sequence,path_position,event_date,touchpoint_date,first_touch_date,last_touch_date
0,nexium24hour-buy.now,1.000000,visit,ppc nonbrand,1,1,only,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58
1,centrum-coupon.submit,0.000047,visit,organic not provided,2,1,introducer,2017-04-21 17:52:51,2017-04-12 12:26:32,2017-04-12 12:26:32,2017-04-21 17:52:51
2,centrum-coupon.submit,0.011891,visit,organic not provided,2,2,closer,2017-04-21 17:52:51,2017-04-21 17:50:56,2017-04-12 12:26:32,2017-04-21 17:52:51
3,centrum-coupon.submit,0.197314,visit,ppc brand,1,1,only,2017-04-30 18:04:52,2017-04-30 18:04:22,2017-04-30 18:04:22,2017-04-30 18:04:52
4,centrum-email.sign.up,1.000000,visit,social network paid,1,1,only,2017-04-19 14:12:59,2017-04-19 14:12:00,2017-04-19 14:12:00,2017-04-19 14:12:00


## procesing

In [37]:
df_model['event_to_touchpoint']=(df_model.event_date-df_model.touchpoint_date).dt.total_seconds()

In [38]:
df_model.event_type.unique()

array(['nexium24hour-buy.now', 'centrum-coupon.submit',
       'centrum-email.sign.up', 'sale.advilpm', 'sale.advil',
       'preparationh-coupon.submit', 'preparationh-buy.now',
       'sale.centrum', 'preparationh-email.sign.up', 'advil pm-buy.now',
       'advil-buy.now', 'centrum-buy.now', 'sale.nexium',
       'emergenc-coupon.submit', 'sale.preparationh',
       'emergenc-email.sign.up', 'emergenc-buy.now', 'sale.emergenc',
       'emergenc-add.to.cart', 'emergenc-freesample.submit',
       'advil-coupon.submit', 'advil-email.sign.up',
       'nexium24hour-coupon.submit', 'nexium24hour-email.sign.up',
       'advil pm-coupon.submit', 'advil pm-email.sign.up',
       'nexium24hour-sweepstakes.social.share',
       'nexium24hour-sweepstakes.submit', 'emergenc-sale.online'], dtype=object)

In [39]:
df_model['day_of_week']=df_model.touchpoint_date.dt.weekday_name

In [40]:
df_model['month']=df_model.touchpoint_date.dt.month

In [41]:
df_model['week_of_month']=(df_model.touchpoint_date.dt.day-1)//7+1

In [42]:
#drop column with date
df_model=df_model.drop(date_col,axis=1)

In [43]:
df_model.head()

,event_type,conversion,touchpoint_type,source_1,touchpoints_in_path,attribution_sequence,path_position,event_to_touchpoint,day_of_week,month,week_of_month
0,nexium24hour-buy.now,1.000000,visit,ppc nonbrand,1,1,only,0.0,Friday,4,1
1,centrum-coupon.submit,0.000047,visit,organic not provided,2,1,introducer,797179.0,Wednesday,4,2
2,centrum-coupon.submit,0.011891,visit,organic not provided,2,2,closer,115.0,Friday,4,3
3,centrum-coupon.submit,0.197314,visit,ppc brand,1,1,only,30.0,Sunday,4,5
4,centrum-email.sign.up,1.000000,visit,social network paid,1,1,only,59.0,Wednesday,4,3


## Get Dummies

In [44]:
df_model_dummies=pd.get_dummies(df_model,columns=['event_type','touchpoint_type','source_1','path_position','day_of_week','month','week_of_month'],
                          drop_first=True)

In [45]:
df_model_dummies.head()

,conversion,touchpoints_in_path,attribution_sequence,event_to_touchpoint,event_type_advil pm-coupon.submit,event_type_advil pm-email.sign.up,event_type_advil-buy.now,event_type_advil-coupon.submit,event_type_advil-email.sign.up,event_type_centrum-buy.now,...,day_of_week_Wednesday,month_2,month_3,month_4,month_11,month_12,week_of_month_2,week_of_month_3,week_of_month_4,week_of_month_5
0,1.000000,1,1,0.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0.000047,2,1,797179.0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,0.011891,2,2,115.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,0.197314,1,1,30.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1.000000,1,1,59.0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0


## Model Creation 

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split,KFold

In [46]:
scaler=StandardScaler()

In [47]:
scaler.fit_transform(df_model_dummies)

array([[ 6.11264301, -0.29509504, -0.25276149, ..., -0.52404415,
        -0.5207466 , -0.31968957],
       [-0.25557136, -0.29175445, -0.25276149, ..., -0.52404415,
        -0.5207466 , -0.31968957],
       [-0.18014268, -0.29175445, -0.24703877, ...,  1.90823618,
        -0.5207466 , -0.31968957],
       ..., 
       [-0.16953274, -0.29509504, -0.25276149, ...,  1.90823618,
        -0.5207466 , -0.31968957],
       [ 4.23897533, -0.29509504, -0.25276149, ..., -0.52404415,
        -0.5207466 ,  3.12803446],
       [ 2.26882468, -0.29509504, -0.25276149, ..., -0.52404415,
        -0.5207466 ,  3.12803446]])

In [51]:
df_model_dummies.head()

,touchpoints_in_path,attribution_sequence,event_to_touchpoint,event_type_advil pm-coupon.submit,event_type_advil pm-email.sign.up,event_type_advil-buy.now,event_type_advil-coupon.submit,event_type_advil-email.sign.up,event_type_centrum-buy.now,event_type_centrum-coupon.submit,...,day_of_week_Wednesday,month_2,month_3,month_4,month_11,month_12,week_of_month_2,week_of_month_3,week_of_month_4,week_of_month_5
0,1,1,0.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2,1,797179.0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,0
2,2,2,115.0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
3,1,1,30.0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
4,1,1,59.0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0


In [50]:
label = df_model_dummies.conversion
df_model_dummies.drop('conversion',axis=1,inplace=True)

In [52]:
df_train,df_test,label_train,label_test = train_test_split(df_model_dummies,label,test_size=.3)

In [53]:
from sklearn import metrics

* Ridge

In [54]:
from sklearn.linear_model import Ridge,Lasso
# predictors=[x for x in train.columns.tolist() if x!=['conversion']]
alg=Ridge(alpha=0.05)

In [55]:
alg.fit(df_train,label_train)

Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [58]:
predict = alg.predict(df_test)

In [59]:
metrics.mean_squared_error(label_test,predict)

0.011945381943696242

* random forest

In [71]:
from sklearn.ensemble import RandomForestRegressor

In [72]:
rf = RandomForestRegressor(n_estimators=50,max_depth=10,random_state=2000)

In [73]:
rf.fit(df_train,label_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=2000,
           verbose=0, warm_start=False)

In [74]:
predict1 = rf.predict(df_test)

In [75]:
metrics.mean_squared_error(label_test,predict1)

0.009202070605306022

In [76]:
pd.DataFrame([label_test.values,predict])

,0,1,2,3,4,5,6,7,8,9,...,308267,308268,308269,308270,308271,308272,308273,308274,308275,308276
0,0.000016,0.000000,0.045442,0.000001,0.006212,0.000436,0.000000,0.000002,0.009634,0.000177,...,0.000004,0.000931,0.000026,0.259669,0.001806,0.000007,0.013259,0.093565,0.007128,0.002351
1,0.122620,0.027876,0.064147,0.003485,0.022338,0.002521,0.005034,-0.007606,-0.011746,-0.004912,...,0.001178,-0.005102,-0.032237,0.355850,-0.004030,0.001360,0.243013,0.541976,-0.003919,0.001642
